In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_constructors
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/constructors.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_constructor (
  CONSTRAINT valid_id EXPECT (constructorId IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_name EXPECT (name IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_nationality EXPECT (nationality IS NOT NULL)
)
COMMENT 'Cleaned constructors data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT constructorId,
       name,
       nationality
  FROM STREAM(bronze_constructors)